### <b>Fastapi</b>
- Fastapi is your ASGI(Asynchronous server gateway interface) supported framework
- Asynchronous programming (async/await) to handle many concurrent requests efficiently.
- Automatic API documentation (SWAGGER http://host:port/docs)
- Pydantic library to automatically validate request and response datas

### <b>Uvicorn</b>
- Uvicorn is the server that acts as a bridge between your fastapi application and the browser
- Uvicorn reads the address(host) door(port) from browser request and delivers the request if person(app) is active(running: uvicorn main:app)
- Browser sends bytes → Uvicorn translates bytes to ASGI → ASGI call is made → FastAPI executes logic → Uvicorn converts ASGI to bytes -> sends bytes back

### <b>python-dotenv</b>
- load_dotenv() injects values into the global OS environment (os.environ) by reading .env file
- dotenv_values() reads .env file and returns a local dictionary with values as key-value pair. When explicit configuration matters
- load_dotenv(find_dotenv()) - Walks up directories, Finds nearest .env

### <b>SQLAlchemy & Drivers (asyncpg, psycopg2 ...)</b>

#### SQLAlchemy

- SQLAlchemy is a package we used as the main center to connect to databases like postgres, mysql databases etc
- SQLAlchemy handles all Connection pooling, Transaction isolation, Concurrency safety, has ORM, used for relational databases.. and many more
- URL -> SQLAlchemy -> Engine -> Pool allocation -> Request arrival -> Engine -> Session -> Database -> Engine -> Close session
- SQLAlchemy does NOT open a database connection when you create the engine. It opens a connection only when the first query actually needs it.
    - Why lazy connections are important ??

        If SQLAlchemy were eager instead of lazy:
        
        - App startup would:
            - Open DB connections immediately
            - Fail if DB is temporarily down
        - You’d waste connections even if:
            - App never hits DB
            - Some routes don’t need DB
- Connections are only created when first query runs, when connections is required.
    - when there is session.execute(query), it says “I actually need a connection → open one now.”
- It will be empty pools if there is no requirements, Engine say “I know how to connect, but I haven’t connected yet.”

#### Driver

- Although we don't manually import drivers anywhere in code, we must need driver to connect to the appropriate databases
    - PostgreSQL (Async) - asyncpg - postgresql+asyncpg://
    - PostgreSQL (Sync) - psycopg2 - postgresql+psycopg2://
    - MySQL (Async) - aiomysql - mysql+aiomysql://
    - MySQL (Sync) - pymysql - mysql+pymysql://
- Its passed along with url, Engine decodes the driver

#### Engine
- Guard of the connection pool
- Manages the connetion pool
- Assigns tokens (session) to new requests.
- if 10 connections are there, each parallel reqeusts are passed to different connections hence different session. 11th parallel request has to wait to get a session until one of them releases their session.
- create engine globally instead of creating in each request inside our application
- Engine creation is expensive, behind the scenes:
    - DNS resolution - parsing the url
    - TCP, TLS handshake
    - Authentication - username password verification etc.
    - session initilization
    - kernel resources
- Engine creation might take around 10-100ms per connection, that X100 request = disaster. so create engine(guard) globally

#### Session

- It begins when a connection is established and ends upon disconnection, log out, or a timeout due to inactivity. 
- How session is created:
    - when you call a request:
        ```bash
        @app.get("/health/db")
        async def db_health_check(db: AsyncSession = Depends(get_db)):
            ...
        ```
    - It calls:
        ``` bash
        async def get_db():
            async with AsyncSessionLocal() as session:
                    yield session
        ```
    - Under the hood (simplified):
        ``` bash
        gen = get_db()
        session = await gen.__anext__()   # before yield
        #route executes here
        await gen.aclose()                # after response
        ```

#### Work flow:

- App starts
- Url read by engine in SQLAlchemy
- <b>Engine</b> created (guard hired)
- Pool initialized (tunnels mapped, not opened yet)
- Request arrives
- <b>Session</b> created (pass issued)
- Connection borrowed (tunnel opened or reused) (managed by Engine(guard))
- Query executed
- Request finishes
- Session closes
- Connection returned to pool
- Engine notes that this connection is free and can be used by others
- App shuts down
- Engine disposed
- All tunnels closed (Guard goes home)

#### Menatl models:

- Engine = connection manager
- Pool = performance optimizer
- Session = request conversation
- Async = concurrency without blocking
- Multiple users ≠ multiple engines
- DB guarantees data safety, not SQLAlchemy